In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import wandb
wandb.login()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit", #worked with unsloth/Meta-Llama-3.1-8B, del 4bit if error
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
INSTRUCTION = "Annotate all entities in the protocol. Only choose one of the following entites: [Action, Reagent, Frequency, Equipment, Labware, Voltage, Mass, Concentration, pH, Volume, Processed, Hint, Method, Time, Waste, Sample, Speed, Temperature, Size]. The output format should look like this {'label': 'Method', 'start': 27, 'end': 40} for each entity, where start is the start character in the sentence and end is the end character of the entity text in the sentence"

prompt = """You are a lab technician analysing molecular biology protocols. Follow the instruction precisely

### Instruction:
{}

### Context:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = INSTRUCTION
    contexts = examples["question"]
    responses = examples["answer"]
    texts = []

    for i,j,k  in zip(instructions, contexts, responses):
        text = prompt.format(i,j,k) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
import pandas as pd
dfTest = pd.read_parquet("/content/drive/MyDrive/BA-File/test.parquet")
dfTrain = pd.read_parquet("/content/drive/MyDrive/BA-File/train.parquet")


In [ ]:
from datasets import Dataset, DatasetDict

#Dataframes
dsTraining = Dataset.from_pandas(dfTrain)
dsValidation = Dataset.from_pandas(dfTest)

#Dicts
dsTrainDict = DatasetDict({"train": dsTraining})
dsValDict = DatasetDict({"test": dsValidation})


In [ ]:
#Mapping
datasets_Train = dsTrainDict.map(formatting_prompts_func, batched = True)
datasets_Test = dsValDict.map(formatting_prompts_func, batched = True)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    evaluation_strategy="no",
    do_eval = False,
    train_dataset = datasets_Train["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 2024,
        output_dir = "Full-Ds-outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:

model.push_to_hub_gguf(repo_id="FynnEnder/FullDs-outputs", tokenizer=tokenizer, quantization_method="q4_k_m")

In [ ]:
Quelle:


Code-Basis von https://github.com/koyeb/example-finetune-llama-3-1/blob/main/notebook.ipynb und für eigene Problemstellung angepasst